<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


In [2]:
# Ta-Libの代替としてpandas_taを利用している。Ta-Libの関数名を小文字にしたものがあるイメージ
!pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=bd72d3718ef1920b7c4bb0e5353da0680b4b54837a065428eb72896198ef7a0c
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [3]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=926109b5539b074b2ad08e36779bc4d67dcec6324255dcd51eb5eebebe4612ce
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


# 銘柄コードから株価情報を取得する関数

In [4]:
from yahoo_finance_api2 import share as yapi2
from yahoo_finance_api2.exceptions import YahooFinanceError
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', 'month', 'week', 'day'）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム（もし株価データが存在しない場合は、None）
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    try:
        data = yapi2.Share(f'{code}.T').get_historical(
            period_type_value,
            period,
            yapi2.FREQUENCY_TYPE_DAY,
            1
        )
    except YahooFinanceError as e:
        print(e.message)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    df = pd.DataFrame(data)

    try:
        # タイムスタンプをDateTime型に変更
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
    except KeyError as e:
        print(e)
        print(f'銘柄コード[{code}]の株価情報を取得できなかったため、スキップしました。')
        return None

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = pd.to_datetime(df['Date'].dt.date)

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

In [ ]:
get_stock_data_yapi2(9501)

,Open,High,Low,Close,Volume
Date,,,,,
2013-06-03,602.0,623.0,586.0,598.0,189300200
2013-06-04,590.0,625.0,582.0,613.0,148783500
2013-06-05,623.0,624.0,513.0,513.0,233103300
2013-06-06,477.0,506.0,451.0,475.0,430786400
2013-06-07,499.0,534.0,485.0,515.0,324742800
...,...,...,...,...,...
2023-05-24,478.0,500.0,477.0,491.0,42155400
2023-05-25,486.0,494.0,479.0,486.0,18880000
2023-05-26,482.0,485.0,477.0,477.0,19031700


# JPX400の銘柄一覧を取得する関数

In [5]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_jpx400():
    """JPX400の銘柄コード一覧を取得する

    :return: JPX400の銘柄コード一覧
    """

    url = "https://indexes.nikkei.co.jp/nkave/archives/file/jpx_nikkei_index_400_weight_jp.csv"
    r = requests.get(url)
    with open('jpx_nikkei_index_400_weight_jp.csv', 'wb') as output:
        output.write(r.content)

    df = pd.read_csv("./jpx_nikkei_index_400_weight_jp.csv", encoding='shift_jis')
    # 不要なレコードとカラムを削除
    df = df.iloc[:-1]   # 最後の行は説明文のため除外
    df = df[['銘柄名', 'コード', '業種']]   # 必要なカラムのみにする
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

In [ ]:
# JPX400の銘柄コードを取得
jpx400 = get_ticker_jpx400()
jpx400

,銘柄名,コード,業種
0,ニッスイ,1332,水産・農林業
1,ウエストホールディングス,1407,建設業
2,ショーボンドホールディングス,1414,建設業
3,ミライト・ワン,1417,建設業
4,ＩＮＰＥＸ,1605,鉱業
...,...,...,...
394,ニトリホールディングス,9843,小売業
395,ミスミグループ本社,9962,卸売業
396,ファーストリテイリング,9983,小売業
397,ソフトバンクグループ,9984,情報・通信業


In [ ]:
# ニッスイの株価情報を取得
get_stock_data_yapi2(jpx400.loc[jpx400['銘柄名'] == 'ニッスイ', 'コード'].values[0])

,Open,High,Low,Close,Volume
Date,,,,,
2013-06-03,193.0,202.0,192.0,192.0,3896600
2013-06-04,197.0,226.0,197.0,212.0,17830600
2013-06-05,205.0,211.0,197.0,198.0,3604200
2013-06-06,198.0,200.0,191.0,192.0,3796300
2013-06-07,190.0,193.0,183.0,191.0,3263300
...,...,...,...,...,...
2023-05-24,637.0,642.0,634.0,636.0,1093600
2023-05-25,634.0,640.0,631.0,636.0,993400
2023-05-26,638.0,638.0,630.0,630.0,762900


# 東証上場の銘柄一覧を取得する関数

In [6]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_tse(segment='prime'):
    """東証上場の銘柄コード一覧を取得する

    :param segment: 東証上場の市場区分（例: 'prime' or 'standard' or 'grows'）
    :return: 東証上場の銘柄コード一覧
    """
    segments = {
        'prime': 'プライム',
        'standard': 'スタンダード',
        'grows': 'グロース'
    }

    if segment not in segments:
        raise Exception("get_ticker_tse関数のsegment引数の値は'prime', 'standard', 'grows'のいずれかにしてください")

    url = "https://www.jpx.co.jp/markets/statistics-equities/misc/tvdivq0000001vg2-att/data_j.xls"
    r = requests.get(url)
    with open('data_j.xls', 'wb') as output:
        output.write(r.content)

    df = pd.read_excel("./data_j.xls")

    # 引数の市場区分のみに絞り込み
    df = df[df['市場・商品区分'].str.startswith(segments[segment])]

    # カラムの削除とカラム名変更
    df = df[['銘柄名', 'コード', '33業種区分']]   # 必要なカラムのみにする
    df = df.rename(columns={'33業種区分': '業種'})
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    return df

In [7]:
# 東証プライム市場の銘柄コードを取得
tse_prime = get_ticker_tse('prime')
tse_prime

,銘柄名,コード,業種
0,極洋,1301,水産・農林業
16,ニッスイ,1332,水産・農林業
17,マルハニチロ,1333,水産・農林業
33,雪国まいたけ,1375,水産・農林業
35,サカタのタネ,1377,水産・農林業
...,...,...,...
4262,サンドラッグ,9989,小売業
4263,サックスバー ホールディングス,9990,小売業
4264,ジェコス,9991,卸売業
4267,グローセル,9995,卸売業


# 株価情報を出来高や株価、パーフェクトオーダーでフィルタリングする関数

In [8]:
import pandas_ta as ta
import math

def is_perfect_order(stock_df, mas, period_type='D', reverse=False):
    """株価情報がパーフェクトオーダーどうかの判定

    :param stock_df: 対象の株価情報のデータフレーム
    :param mas: 移動平均線に使用するためのリスト（例: [5, 25, 75]）
    :param period_type: 期間をどの単位にするか（'W' or 'M'）
    :param reverse: 下降トレンドのパーフェクトオーダーを検出する場合は、True / 上昇トレンドの場合は、False
    :return: 株価情報がパーフェクトオーダーの場合は、True / それ以外の場合は、False
    """

    df = stock_df.copy()

    # 月足、周足にリサンプリング
    if period_type in ('M', 'W'):
        resampled = df.resample(period_type)
        df = resampled.aggregate({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'})
    
    close = df['Close']
    
    # 移動平均線の算出
    for ma in mas:
        df[f'ma{ma}{period_type}'] = ta.sma(close, ma)
    
    # パーフェクトオーダーかどうかの判定
    conditions = []
    try:
        if not reverse:
            # 上昇トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) >= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
        else:
            # 下降トレンド
            for i in range(len(mas) - 1):
                conditions.append(math.ceil(df[f'ma{mas[i]}{period_type}'].values[-1]) <= math.ceil(df[f'ma{mas[i + 1]}{period_type}'].values[-1]))
    except TypeError:
        # 株価情報が少なくて移動平均線が取得できない場合は、無条件でFalseでリターンする
        return False

            
    result = all(conditions)

    return result

In [9]:
def filtered_ticker(df, min_volume=None, max_price=None, perfect_order={}):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :param perfect_order: パーフェクトオーダーの判定に使用する辞書（例: {'mas': [5, 20, 60], 'period_type': 'D', 'reverse': False}）（辞書のキー'mas'のみ必須）
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    # 空のDataFrameを作成
    result_df = pd.DataFrame()

    # 各銘柄についてループ
    for index, row in df.iterrows():
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        if not perfect_order:
            stock_data = get_stock_data_yapi2(code, period_type='day', period=2)
        else:
            largest_mas = perfect_order.get('mas')[-1]
            period_type = perfect_order.get('period_type', 'D')
            if period_type == 'D':
                period = math.ceil(largest_mas / 15)
                stock_data = get_stock_data_yapi2(code, period_type='month', period=period)
            elif period_type == 'W':
                period = math.ceil(largest_mas * 7 / 365)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            elif period_type == 'M':
                period = math.ceil(largest_mas / 10)
                stock_data = get_stock_data_yapi2(code, period_type='year', period=period)
            else:
                raise ValueError('引数perfect_orderは"D"か"W"か"M"のいずれかを指定してください')
        # 株価情報が取得できなかったら、次の銘柄コードに遷移
        if stock_data is None:
            continue

        # 出来高の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        if min_volume is not None and volume < min_volume:
            continue
        
        # 価格の条件に合わない場合、次の銘柄コードに遷移
        price = stock_data['Close'][-1]
        if max_price is not None and price > max_price:
            continue
        
        # パーフェクトオーダーの条件に合わない場合、次の銘柄コードに遷移
        if perfect_order:
            mas = perfect_order.get('mas')
            period_type = perfect_order.get('period_type', 'D')
            reverse = perfect_order.get('reverse', False)
            if not is_perfect_order(stock_data, mas=mas, period_type=period_type, reverse=reverse):
                continue

        # 銘柄名と業種を設定
        filtered_df = pd.DataFrame({
            '銘柄名': [row['銘柄名']],
            'コード': [code],
            '業種': [row['業種']],
            '出来高': [volume],
            '株価': [price],
        }, index=[index])

        # 結果をresult_dfに追加
        result_df = pd.concat([result_df, filtered_df])

    return result_df

# JPX400をフィルタリング


In [ ]:
# JPX400の銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_jpx400 = filtered_ticker(jpx400, min_volume=200000, max_price=5000)
filtered_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,706000,634.0
1,ウエストホールディングス,1407,建設業,269800,2779.0
4,ＩＮＰＥＸ,1605,鉱業,4520200,1527.0
5,安藤・間,1719,建設業,620900,1022.0
6,コムシスホールディングス,1721,建設業,406600,2758.0
...,...,...,...,...,...
380,大阪瓦斯,9532,電気・ガス業,612300,2252.0
382,エヌ・ティ・ティ・データ,9613,情報・通信業,2046000,2026.0
387,ＳＣＳＫ,9719,情報・通信業,436200,2229.0
395,ミスミグループ本社,9962,卸売業,1032600,3145.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_jpx400 = filtered_ticker(filtered_jpx400, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,706000,634.0
5,安藤・間,1719,建設業,620900,1022.0
6,コムシスホールディングス,1721,建設業,406600,2758.0
7,大成建設,1801,建設業,761900,4545.0
8,大林組,1802,建設業,1938400,1171.0
...,...,...,...,...,...
373,関西電力,9503,電気・ガス業,1893800,1614.0
375,北海道電力,9509,電気・ガス業,562200,571.0
379,東京瓦斯,9531,電気・ガス業,1093900,3010.0
395,ミスミグループ本社,9962,卸売業,1032600,3145.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_jpx400 = filtered_ticker(filtered_perfect_order_week_jpx400,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,706000,634.0
5,安藤・間,1719,建設業,620900,1022.0
6,コムシスホールディングス,1721,建設業,406600,2758.0
8,大林組,1802,建設業,1938400,1171.0
9,清水建設,1803,建設業,1855800,863.0
...,...,...,...,...,...
372,中部電力,9502,電気・ガス業,1018200,1669.0
373,関西電力,9503,電気・ガス業,1893800,1614.0
375,北海道電力,9509,電気・ガス業,562200,571.0
379,東京瓦斯,9531,電気・ガス業,1093900,3010.0


# 東証プライムをフィルタリング

In [ ]:
# 東証プライムの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_prime = filtered_ticker(tse_prime, min_volume=200000, max_price=5000)
filtered_tse_prime

Not Found: No data found, symbol may be delisted
銘柄コード[25935]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[8096]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,706000,634.0
35,サカタのタネ,1377,水産・農林業,397500,3940.0
61,タマホーム,1419,建設業,2538800,3630.0
64,ファーストコーポレーション,1430,建設業,299300,825.0
69,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,335400,183.0
...,...,...,...,...,...
4191,ヤマダホールディングス,9831,小売業,3241000,417.0
4194,モリト,9837,卸売業,390200,1020.0
4246,ミスミグループ本社,9962,卸売業,1032600,3145.0
4261,スズケン,9987,卸売業,207300,3525.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,706000,634.0
61,タマホーム,1419,建設業,2538800,3630.0
64,ファーストコーポレーション,1430,建設業,299300,825.0
69,Ｒｏｂｏｔ Ｈｏｍｅ,1435,建設業,335400,183.0
223,安藤・間,1719,建設業,620900,1022.0
...,...,...,...,...,...
4062,東京瓦斯,9531,電気・ガス業,1093900,3010.0
4164,オオバ,9765,サービス業,299300,753.0
4194,モリト,9837,卸売業,390200,1020.0
4246,ミスミグループ本社,9962,卸売業,1032600,3145.0


In [ ]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_prime = filtered_ticker(filtered_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
16,ニッスイ,1332,水産・農林業,706000,634.0
61,タマホーム,1419,建設業,2538800,3630.0
223,安藤・間,1719,建設業,620900,1022.0
225,コムシスホールディングス,1721,建設業,406600,2758.0
240,ワイ・ティー・エル・コーポレーション・バーハッド,1773,建設業,774000,29.0
...,...,...,...,...,...
4052,東北電力,9506,電気・ガス業,1822100,826.0
4053,四国電力,9507,電気・ガス業,641900,950.0
4055,北海道電力,9509,電気・ガス業,562200,571.0
4062,東京瓦斯,9531,電気・ガス業,1093900,3010.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_prime = filtered_ticker(filtered_tse_prime, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_prime

,銘柄名,コード,業種,出来高,株価
266,三井住友建設,1821,建設業,961600,359.0
392,日本Ｍ＆Ａセンターホールディングス,2127,サービス業,3277000,997.0
412,パソナグループ,2168,サービス業,224800,1759.0
414,リンクアンドモチベーション,2170,サービス業,458900,416.0
460,明治ホールディングス,2269,食料品,723400,3170.0
...,...,...,...,...,...
4060,レノバ,9519,電気・ガス業,769600,1624.0
4088,エイチ・アイ・エス,9603,サービス業,566300,1941.0
4093,共立メンテナンス,9616,サービス業,608800,4970.0
4173,ベネッセホールディングス,9783,サービス業,478100,1863.0


In [ ]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_prime = filtered_ticker(filtered_reverse_perfect_order_week_tse_prime,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_prime

,銘柄名,コード,業種,出来高,株価
266,三井住友建設,1821,建設業,961600,359.0
412,パソナグループ,2168,サービス業,224800,1759.0
414,リンクアンドモチベーション,2170,サービス業,458900,416.0
460,明治ホールディングス,2269,食料品,723400,3170.0
492,いちご,2337,不動産業,943400,248.0
...,...,...,...,...,...
4060,レノバ,9519,電気・ガス業,769600,1624.0
4088,エイチ・アイ・エス,9603,サービス業,566300,1941.0
4093,共立メンテナンス,9616,サービス業,608800,4970.0
4173,ベネッセホールディングス,9783,サービス業,478100,1863.0


# 東証スタンダードをフィルタリング

In [10]:
# 東証スタンダード市場の銘柄コードを取得
tse_standard = get_ticker_tse('standard')
tse_standard

,銘柄名,コード,業種
34,カネコ種苗,1376,水産・農林業
37,秋川牧園,1380,水産・農林業
38,アクシーズ,1381,水産・農林業
39,ホーブ,1382,水産・農林業
40,ベルグアース,1383,水産・農林業
...,...,...,...
4257,タキヒヨー,9982,卸売業
4260,蔵王産業,9986,卸売業
4265,ヤマザワ,9993,小売業
4266,やまや,9994,小売業


In [11]:
# 東証スタンダードの銘柄コードを以下の条件に合致するもののみにフィルタリング
# 出来高　... 20万株以上
# 株価 ... 5000円以下
filtered_tse_standard = filtered_ticker(tse_standard, min_volume=200000, max_price=5000)
filtered_tse_standard

'timestamp'
銘柄コード[2777]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9318]の株価情報を取得できなかったため、スキップしました。
'timestamp'
銘柄コード[9734]の株価情報を取得できなかったため、スキップしました。


,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,269800,2779.0
120,住石ホールディングス,1514,鉱業,457100,348.0
233,中小企業ホールディングス,1757,建設業,1646800,48.0
244,アジアゲートホールディングス,1783,建設業,612700,52.0
385,塩水港精糖,2112,食料品,469100,204.0
...,...,...,...,...,...
4006,ビート・ホールディングス・リミテッド,9399,情報・通信業,5656612,12.0
4030,エム・エイチ・グループ,9439,サービス業,1213100,250.0
4132,アゴーラ ホスピタリティー グループ,9704,サービス業,592800,25.0
4183,ストライダーズ,9816,不動産業,341100,244.0


In [12]:
# 株価情報が上昇トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W'})
filtered_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
233,中小企業ホールディングス,1757,建設業,1646800,48.0
386,フジ日本精糖,2114,食料品,453900,652.0
408,ｎｍｓ ホールディングス,2162,サービス業,459400,501.0
474,ＣＳＳホールディングス,2304,サービス業,468100,925.0
494,極楽湯ホールディングス,2340,サービス業,299200,311.0
522,鉄人化計画,2404,サービス業,3202200,385.0
537,シダー,2435,サービス業,1412500,278.0
966,白鳩,3192,小売業,251700,365.0
986,プロパスト,3236,不動産業,245300,160.0
1036,ミタチ産業,3321,卸売業,229100,1328.0


In [13]:
# 株価情報が上昇トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_perfect_order_week_day_tse_standard = filtered_ticker(filtered_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60]})
filtered_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
233,中小企業ホールディングス,1757,建設業,1646800,48.0
386,フジ日本精糖,2114,食料品,453900,652.0
474,ＣＳＳホールディングス,2304,サービス業,468100,925.0
494,極楽湯ホールディングス,2340,サービス業,299200,311.0
522,鉄人化計画,2404,サービス業,3202200,385.0
537,シダー,2435,サービス業,1412500,278.0
1036,ミタチ産業,3321,卸売業,229100,1328.0
1190,クラウディアホールディングス,3607,繊維製品,211100,561.0
1257,ジェクシード,3719,情報・通信業,266800,236.0
1413,ＰＫＳＨＡ Ｔｅｃｈｎｏｌｏｇｙ,3993,情報・通信業,3387300,2758.0


In [14]:
# 株価情報が下降トレンドのパーフェクトオーダー（週足）かどうかの判断
filtered_reverse_perfect_order_week_tse_standard = filtered_ticker(filtered_tse_standard, perfect_order={'mas': [5, 20, 60], 'period_type': 'W', 'reverse': True})
filtered_reverse_perfect_order_week_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,269800,2779.0
244,アジアゲートホールディングス,1783,建設業,612700,52.0
479,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1046100,51.0
498,クシム,2345,情報・通信業,201000,387.0
539,Ｓｈｉｎｗａ Ｗｉｓｅ Ｈｏｌｄｉｎｇｓ,2437,サービス業,308900,557.0
560,出前館,2484,情報・通信業,794900,440.0
666,ベクター,2656,小売業,5049700,345.0
705,ピクセルカンパニーズ,2743,卸売業,371300,45.0
941,大光,3160,卸売業,238000,591.0
1046,メタプラネット,3350,卸売業,1375100,23.0


In [16]:
# 株価情報が下降トレンドのパーフェクトオーダー（日足）かどうかの判断
filtered_reverse_perfect_order_week_day_tse_standard = filtered_ticker(filtered_reverse_perfect_order_week_tse_standard,  perfect_order={'mas': [5, 20, 60], 'reverse': True})
filtered_reverse_perfect_order_week_day_tse_standard

,銘柄名,コード,業種,出来高,株価
57,ウエストホールディングス,1407,建設業,269800,2779.0
244,アジアゲートホールディングス,1783,建設業,612700,52.0
479,ＣＡＩＣＡ ＤＩＧＩＴＡＬ,2315,情報・通信業,1046100,51.0
498,クシム,2345,情報・通信業,201000,387.0
1046,メタプラネット,3350,卸売業,1375100,23.0
1281,ブロードバンドタワー,3776,情報・通信業,822700,139.0
1299,サイバーステップ,3810,情報・通信業,231800,462.0
1305,リミックスポイント,3825,情報・通信業,558000,248.0
1470,田中化学研究所,4080,化学,241600,1221.0
2181,エス・サイエンス,5721,非鉄金属,305400,24.0
